# 데이터 전처리

In [2]:
import pandas as pd
import re
from konlpy.tag import Okt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk import pos_tag
from collections import Counter
import pickle

decades = ['1960s', '1970s', '1980s', '1990s', '2000s', '2010s', '2020s']

##1. 시대별 파일을 가져와서 각각의 변수에 저장 / 결측치 제거 + 말뭉치 생성 / 리스트 형태로 변수에 저장
def load_lyrics(decades):
    lyrics_1960s = pd.read_csv("crawling/1960s.csv")['가사'].dropna().tolist()
    lyrics_1970s = pd.read_csv("crawling/1960s.csv")['가사'].dropna().tolist()
    lyrics_1980s = pd.read_csv("crawling/1960s.csv")['가사'].dropna().tolist()
    lyrics_1990s = pd.read_csv("crawling/1960s.csv")['가사'].dropna().tolist()
    lyrics_2000s = pd.read_csv("crawling/1960s.csv")['가사'].dropna().tolist()
    lyrics_2010s = pd.read_csv("crawling/1960s.csv")['가사'].dropna().tolist()
    lyrics_2020s = pd.read_csv("crawling/1960s.csv")['가사'].dropna().tolist()

    return lyrics_1960s, lyrics_1970s, lyrics_1980s, lyrics_1990s, lyrics_2000s, lyrics_2010s, lyrics_2020s

# 불용어 설정
stop_words_nltk = set(nltk_stopwords.words('english'))
stop_words_nltk.update(['oh', 'uh', 'u', 'na', 'cant', 'dont', 'wan', 'j', 'tu', 'yo', 'la', 'al', 'el', 'hey', 'un', 'ta', 'uaagh', 'aint', 'bam', 'im', 'ah', 'e', 'eh', 'ooh', 'yeah', 'ya', 'woo', 'ba', 'du', 'ti', 'gon', 'imma', 'da']) # 불용어 추가
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    custom_stop_words = f.read().splitlines()
stop_words = stop_words_nltk.union(set(custom_stop_words))

# 영어 품사 태깅 용어 설정
def get_wordnet_tagset(tag):
    if tag.startswith('N'):
        return 'n'
    elif tag.startswith('V'):
        return 'v'
    elif tag.startswith('J'):
        return 'a'
    elif tag.startswith('R'):
        return 'r'
    else:
        return None

# 정규 표현식 영어 구분, 한국어/영어 토큰화, 품사태깅 및 원형 복원
def preprocess_text(text, stop_words):
    ko_nouns, ko_adj, en_nouns, en_adj = [], [], [], []

    # 1) 정규 표현식으로 한글/영어 구분 및 특수문자 제거
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')
    english = re.compile('[^a-zA-Z\s]')
    result_en = english.sub('', text).lower() # 소문자화
    result_kr = hangul.sub('', text)

    # 2) 영어 토큰화 및 품사 태깅, 원형 복원, 불용어 적용
    en_tokens = word_tokenize(result_en) # 토큰화
    en_pos_list = pos_tag(en_tokens) # 품사 태깅
    
    for word, tag in en_pos_list:
        # 불용어 설정 및 명사 품사 반환
        if get_wordnet_tagset(tag) == 'n' and word.lower() not in stop_words:
            en_nouns.append(WordNetLemmatizer().lemmatize(word.lower(), pos='n'))
        # 불용어 설정 및 형용사 품사 반환
        elif get_wordnet_tagset(tag) == 'a' and word.lower() not in stop_words:
            en_adj.append(WordNetLemmatizer().lemmatize(word.lower(), pos='a'))

    # 3) 한국어 토큰화 및 품사 태깅, 원형 복원, 불용어 적용
    okt = Okt() # okt활용
    ko_pos_list = okt.pos(result_kr, stem=True) # 품사 태깅 및 원형 복원
    
    for word, pos in ko_pos_list:
        # 불용어 설정 및 명사 품사 반환
        if pos == 'Noun' and word not in stop_words:
            ko_nouns.append(word)
        # 불용어 설정 및 형용사 품사 반환
        elif pos == 'Adjective' and word not in stop_words:
            ko_adj.append(word)

    return ko_nouns, ko_adj, en_nouns, en_adj

def preprocess_data(lyrics_data, stop_words):
    # word_counters 딕셔너리 생성
    word_counters = {decade: {'ko_nouns': Counter(), 'ko_adj': Counter(), 'en_nouns': Counter(), 'en_adj': Counter()} for decade in decades}
    # decades 리스트와 lyrics_data 리스트의 각 항목을 동시에 순회하며 decades가 속하는 가사데이터 처리
    for decade, lyrics in zip(decades, lyrics_data):
        for text in lyrics:
            ko_nouns, ko_adj, en_nouns, en_adj = preprocess_text(text, stop_words)
            
            word_counters[decade]['ko_nouns'].update(ko_nouns)
            word_counters[decade]['ko_adj'].update(ko_adj)
            word_counters[decade]['en_nouns'].update(en_nouns)
            word_counters[decade]['en_adj'].update(en_adj)

    return word_counters

# word_counters를 데이터 프레임으로 변환 및 저장
def save_to_csv(word_counters):
    #데이터 프레임 초기화
    df_rows = []

    for decade, counters in word_counters.items(): # decades 딕셔너리의 키-값 쌍 순회
        for pos, counter in counters.items(): # 빈도수 딕셔너리의 키-값 쌍 순회
            language = 'ko' if 'ko' in pos else 'en'
            for word, freq in counter.items(): #counter 객체의 키-값 쌍 순회
                df_rows.append([decade, word, pos, language, freq])
    
    df = pd.DataFrame(df_rows, columns=['연대', '단어', '품사', '언어', '빈도수'])
    df.to_csv('dataframe/yearly_lyrics.csv', index=False, encoding='utf-8-sig')

# 데이터 로딩
lyrics_data = load_lyrics(decades)

# 데이터 전처리
word_counters = preprocess_data(lyrics_data, stop_words)

# 데이터 저장
save_to_csv(word_counters)

# 시각화_WordCloud

In [14]:
# WordCloud
from wordcloud import WordCloud
import numpy as np
from PIL import Image

# #깃허브로부터 나눔폰트 다운로드
# ! git clone https://github.com/namepen/nanum_font.git

# 데이터 로딩
df = pd.read_csv('dataframe/yearly_lyrics.csv')

# 단어+빈도수 -> vocab으로 만들시 품사, 언어별 wordcloud를 만들기 어려움 
def words_counts(df) -> dict:
    d = df[['단어', '빈도수']]
    d.index = d['단어']
    d = d.drop(['단어'], axis=1)
    return d.to_dict()['빈도수']

def save_wc(filename: str, data: dict, max_font_size=150,  width=512, height=512):
        
    masking_image = np.array(Image.open("images/apple.jpeg"))
    wc = WordCloud(
        font_path= "c:\\classes\\classes\\06_NLP_Preporcessing\\nanum_font\\NanumGothic-ExtraBold.ttf", 
        background_color="white", 
        max_font_size=max_font_size, 
        width=width, height=height, 
        mask=masking_image)
    
    cloud = wc.generate_from_frequencies(data)
    file_path=f"wordcloud/{filename}.png"
    cloud.to_file(file_path)
    return cloud

decades = ['1960s', '1970s', '1980s', '1990s', '2000s', '2010s', '2020s']

for decade in decades:
    ko = (df['언어'] == 'ko') & (df['연대'] == decade)
    en = (df['언어'] == 'en') & (df['연대'] == decade)
    ko_noun = (df['품사'] == 'ko_nouns') & (df['연대'] == decade)
    ko_adj = (df['품사'] == 'ko_adj') & (df['연대'] == decade)
    en_noun = (df['품사'] == 'en_nouns') & (df['연대'] == decade)
    en_adj = (df['품사'] == 'en_adj') & (df['연대'] == decade)

    # 전체 명사
    ko_noun_df = df[ko_noun]
    en_noun_df = df[en_noun]

    all_noun_df = pd.concat([ko_noun_df, en_noun_df])
    save_wc(f'{decade}_all_noun', words_counts(all_noun_df))

    # 전체 형용사
    ko_adj_df=df[ko_adj]
    en_adj_df=df[en_adj]

    all_adj_df=pd.concat([ko_adj_df,en_adj_df])
    save_wc(f'{decade}s_all_adj', words_counts(all_adj_df))

    # 한글
    kr_df = df[ko]
    save_wc(f'{decade}s_kr', words_counts(kr_df))

    # 한글 + 명사
    kr_noun = df[ko & ko_noun]
    save_wc(f'{decade}s_kr_noun', words_counts(kr_noun))

    # 한글 + 형용사
    kr_adj = df[ko & ko_adj]
    save_wc(f'{decade}s_kr_adj', words_counts(kr_adj))

    # 영어
    en_df = df[en]
    save_wc(f'{decade}s_en', words_counts(en_df))

    # 영어 + 명사
    en_noun = df[en & en_noun]
    save_wc(f'{decade}s_en_noun', words_counts(en_noun))

    # 영어 + 형용사
    en_adj = df[en & en_adj]
    save_wc(f'{decade}s_en_adj', words_counts(en_adj))

ImportError: cannot import name 'WordCloud' from 'wordcloud' (unknown location)

# 시각화_ 시대별 Top20 단어 

In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'

# 데이터 로딩
df = pd.read_csv('dataframe/yearly_lyrics.csv')

# 각 연대별로 필터링
def filter_top_words(df, decade, pos, top=20):
    return df[(df['연대'] == decade) & (df['품사'] == pos)].nlargest(top, '빈도수')

def plot_top_words(df, title):
    plt.figure(figsize=(12, 6))
    plt.barh(df['단어'], df['빈도수'], color='skyblue')
    plt.xlabel('빈도수')
    plt.ylabel('단어')
    plt.title(title)
    plt.gca().invert_yaxis()  # 빈도수가 높은 단어부터 표시
    plt.tight_layout()

    # 그래프를 이미지 파일로 저장
    save_path = os.path.join('topbar', f'{title}.png')
    plt.savefig(save_path)
    plt.close()

decades = ['1960s', '1970s', '1980s', '1990s', '2000s', '2010s', '2020s']

def save_plots(decades):
    if not os.path.exists('topbar'):
        os.makedirs('topbar')
        
    for decade in decades:
        df_ko_nouns = filter_top_words(df, decade, 'ko_nouns')
        df_ko_adj = filter_top_words(df, decade, 'ko_adj')
        df_en_nouns = filter_top_words(df, decade, 'en_nouns')
        df_en_adj = filter_top_words(df, decade, 'en_adj')
        
        plot_top_words(df_ko_nouns, f'{decade} 한국어 명사 Top 20')
        plot_top_words(df_ko_adj, f'{decade} 한국어 형용사 Top 20')
        plot_top_words(df_en_nouns, f'{decade} 영어 명사 Top 20')
        plot_top_words(df_en_adj, f'{decade} 영어 형용사 Top 20')

save_plots(decades)

# 시각화_ 시대별 영어 한국어 비중

In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'

# 데이터 로딩
df = pd.read_csv('dataframe/yearly_lyrics.csv')

# 누적 막대 그래프를 그리는 함수
def plot_stacked_bar(df, title):
    # '연대' 컬럼을 기준으로 그룹화
    grouped = df.groupby(['연대', '단어']).sum().reset_index()

    # 영어와 한국어 빈도를 나타내는 컬럼 생성
    grouped['ko'] = grouped.apply(lambda row: row['빈도수'] if row['언어'] == 'ko' else 0, axis=1)
    grouped['en'] = grouped.apply(lambda row: row['빈도수'] if row['언어'] == 'en' else 0, axis=1)

    # '연대'로 다시 그룹화하여 누적 막대 그래프를 위한 데이터 생성
    stacked = grouped.groupby('연대').sum().reset_index()

    # 누적 막대 그래프 그리기
    plt.figure(figsize=(12, 6))
    barWidth = 0.85

    r = np.arange(len(stacked['연대']))

    plt.bar(r, stacked['ko'], color='#b5ffb9', edgecolor='white', width=barWidth, label='Korean')
    plt.bar(r, stacked['en'], bottom=stacked['ko'], color='#f9bc86', edgecolor='white', width=barWidth, label='English')

    plt.xlabel('Decades', fontweight='bold')
    plt.xticks([r for r in range(len(stacked['연대']))], stacked['연대'])
    plt.ylabel('Frequency', fontweight='bold')
    plt.title(title)
    plt.legend()

    plt.savefig(f'krenbar/{title}.png')
    plt.close()

# 누적 막대 그래프 그리기
plot_stacked_bar(df, 'Language Comparison by Decades')

# DF-IDF

In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from konlpy.tag import Okt
import matplotlib.pyplot as plt
import numpy as np
import re

# 한국어 깨짐 방지
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# 한국어 불용어
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    kr_stopwords = [line.strip() for line in f.readlines()]

okt = Okt()
lemmatizer = WordNetLemmatizer()

# 한국어 전처리 함수
def preprocess_kor_nouns(text):
    ko_text = re.sub(r'[^ㄱ-ㅎ가-힣\s]+', '', text).strip()
    tagged_words = okt.pos(ko_text, stem=True)
    filtered_words = [word for word, tag in tagged_words if tag in (['Noun','Adjective']) and word not in kr_stopwords]
    return ' '.join(filtered_words)

decades = ['1960s', '1970s', '1980s', '1990s', '2000s', '2010s', '2020s']

tfidf_df = pd.DataFrame()

for decade in decades:

    # 파일 읽기
    df = pd.read_csv(f'crawling/{decade}.csv')
    df['가사'] = df['가사'].fillna('')  # 결측값 처리

    # 영어와 한국어 분리 -> 전처리
    df['lyrics_kr'] = df['가사'].apply(preprocess_kor_nouns)

    # 한국어 TF-IDF 변환
    tfidf_vectorizer_kr = TfidfVectorizer(max_features=20_000, min_df=5)
    tfidf_matrix_kr = tfidf_vectorizer_kr.fit_transform(df['lyrics_kr'])
    feature_names_kr = tfidf_vectorizer_kr.get_feature_names_out()

    # 각 연대별 TF-IDF 값을 Series로 변환하여 DataFrame에 추가
    tfidf_series = pd.Series(tfidf_matrix_kr.toarray().sum(axis=0), index=feature_names_kr, name=decade).sort_values(ascending=False)
    tfidf_df = pd.concat([tfidf_df, tfidf_series], axis=1)

# 결과 확인
tfidf_df.head()

# 파일 저장
file_path=f"count_info/tfidf.csv"
tfidf_df.to_csv(file_path)